In [ ]:
# 1. Rewindow button seems to make image very bright
# 2. move slider with mouse

In [2]:
import os
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button

## Flash simulator

This simulator loads T1, T2 and PD parameter maps, visualizes them, and then runs the simulation to construct an estimated MR image based on the users input of TR, TE and Flip Angle (FA) for a chosen slice.

The simulator implements the following equation:

$$
     Mxy = \text{PD}*\frac{(1-e^{-\text{TR}/\text{T1}})}{(1-\cos{(FA)}*e^{-TR/T1})}* \sin{(FA)}*e^{-\text{TE}/\text{T2}}
$$
<br>
*Code History:*<br>
- *CP:  October 2021* <br>
&emsp; First Python version <br><br>
- *PJV: v.1.4: November 2017* <br>
   &emsp; Changes: - It loads the parameter maps directly, rather than computing them from the original images. It makes it                less confusing. <br>
   &emsp;&emsp;&emsp;&emsp;&emsp; $\;\;\;$ - If the script fails to open any of the image files it returns an error asking the user to cd to the                  mriTools/mriTutorial folder.<br><br>
- *PJV: v.1.3: December 2015* <br>
   &emsp; Minor change: - limit the minimum FA to 10 deg<br><br>
- *PJV: v.1.2: November 2013*<br>
    &emsp;Minor changes: - limit the minimum TR to TE+1 ms.<br>
    &emsp;&emsp;&emsp;&emsp;&emsp; $\;\;\;\;\;\;\;\;\;\;$- increase the minimum TR and TE to 1 ms.<br>
    &emsp;&emsp;&emsp;&emsp;&emsp; $\;\;\;\;\;\;\;\;\;\;$- improve functionality of the "window" button.<br><br>
- PJV: v.1.1: August 2011 <br>
    &emsp;Minor changes: - use "single" variables, for better memory performance.<br>
    &emsp;&emsp;&emsp;&emsp;&emsp; $\;\;\;\;\;\;\;\;\;\;$- only update the image when there have been changes in the parameter values.<br> <br>
- *PJV: v.1.0: March 2009* <br>
     &emsp;Created from MR_simulator.m (Written by G.M. Boynton for Psych 555C, Spring 2007 at U. Washington) Tissue parameters are based on the McGill &emsp;brain database (http://www.bic.mni.mcgill.ca/brainweb/tissue_mr_parameters.txt)



### 1. Load parameter maps and show T1, T2 and PD images in a figure

In [25]:
#Path for data
mri_data_path = os.getcwd() + '/data/mri_data/'

In [26]:
#Load T1, T2, and PD

T1 = scipy.io.loadmat(mri_data_path+'T1.mat')['T1']
T2 = scipy.io.loadmat(mri_data_path+'T2.mat')['T2']
PD = scipy.io.loadmat(mri_data_path+'PD.mat')['PD']

In [27]:
# Check their sizes
print(T1.shape)
print(T2.shape)
print(PD.shape)

(181, 217, 181)
(181, 217, 181)
(181, 217, 181)


Let's pick an axial slice and show it:

In [73]:
z = 99

#Plot it
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=[20,10])
fig.subplots_adjust(wspace=0.04)

#T1
ax0 = axes[0].imshow(np.flipud(np.transpose(PD[:,:,z])/np.transpose(T1[:,:,z])), cmap='Greys_r')
ax0.set_clim(0, 0.0015) 
axes[0].set_title('T1', fontsize = 20, fontweight='bold', fontname='Arial')
axes[0].axis('off')

#T2
ax1 = axes[1].imshow(np.flipud(np.transpose(T2[:,:,z])), cmap='Greys_r')
ax1.set_clim(50, 120) 
axes[1].set_title('T2', fontsize = 20, fontweight='bold', fontname='Arial')
axes[1].axis('off')

#PD
ax2 = axes[2].imshow(np.flipud(np.transpose(PD[:,:,z])), cmap='Greys_r')
axes[2].set_title('PD', fontsize = 20, fontweight='bold', fontname='Arial')
axes[2].axis('off')

plt.show()

### 2. Set up the simulator

In [75]:
%matplotlib qt 

In [76]:
# The parametrized function to be plotted
def f(PD, TR, T1, FA, TE, T2, z):
    Mxy = PD[:,:,z]*(1-np.exp(-TR/T1[:,:,z]))/(1-np.cos(np.deg2rad(FA))*np.exp(-TR/T1[:,:,z]))* np.sin(np.deg2rad(FA))*np.exp(-TE/T2[:,:,z])
    Mxy = np.flipud(np.transpose(Mxy))
    return 64*Mxy/np.max(Mxy[:])

#Initial parameters:
TR = 500  #in msec
TE = 15   #in msec
FA = 90 #in degrees
z = 99

# Create the figure 
fig, ax = plt.subplots(figsize = (15,15))
simulator = ax.imshow(f(PD, TR, T1, FA, TE, T2, z), cmap='Greys_r')
ax.axis('off')
axcolor = 'silver'
ax.margins(x=0)
# adjust the main plot to make room for the sliders
plt.subplots_adjust(left=0.1, bottom=0.2)

# Make a horizontal slider to control the FA.
ax_FA = plt.axes([0.3, 0.03, 0.4, 0.03], facecolor=axcolor)
FA_slider = Slider(
    ax=ax_FA,
    label='FA    ',
    valmin=10,
    valmax=180,
    valstep = 1,
    valinit=FA,
    valfmt='    %i'
)


# Make a horizontal slider to control the TR.
ax_TR = plt.axes([0.3, 0.07, 0.4, 0.03], facecolor=axcolor)
TR_slider = Slider(
    ax=ax_TR,
    label='TR    ',
    valmin=1,
    valmax=2000,
    valstep = 1,
    valinit=TR,
    valfmt='    %i'
)

# Make a horizontal slider to control the TE.
ax_TE = plt.axes([0.3, 0.11, 0.4, 0.03], facecolor=axcolor)
TE_slider = Slider(
    ax=ax_TE,
    label='TE    ',
    valmin=1,
    valmax=100,
    valstep = 1,
    valinit=TE,
    valfmt='    %i'
)

# Make a horizontal slider to control the slice z.
ax_slice = plt.axes([0.3, 0.15, 0.4, 0.03], facecolor=axcolor)
slice_slider = Slider(
    ax=ax_slice,
    label='Slice    ',
    valmin=0,
    valmax=180,
    valstep = 1,
    valinit=z,
    valfmt='    %i'
)


# The function to be called anytime a slider's value changes
def update(val):
    if int(TR_slider.val) < int(TE_slider.val):
        TR_slider.set_val(int(TE_slider.val)+1)
    simulator.set_data(f(PD, int(TR_slider.val), T1, FA_slider.val, int(TE_slider.val), T2, int(slice_slider.val)))
    fig.canvas.draw_idle()
    
    
# Register the update function with each slider
TR_slider.on_changed(update)
TE_slider.on_changed(update)
slice_slider.on_changed(update)
FA_slider.on_changed(update)

# Create a `matplotlib.widgets.Button` to increase the FA slider value (by 2 each time)
FA_right_button_ax = plt.axes([0.7, 0.03, 0.014, 0.03])
FA_right_button = Button(FA_right_button_ax, '>', color=axcolor, hovercolor='0.975')
def increase_FA(event):
    if FA_slider.val in range(10,179):
        FA_slider.set_val(FA_slider.val+2)
        simulator.set_data(f(PD, int(TR_slider.val), T1, FA_slider.val, int(TE_slider.val), T2, int(slice_slider.val)))
    elif FA_slider.val ==179:
        FA_slider.set_val(180)
        simulator.set_data(f(PD, int(TR_slider.val), T1, 180, int(TE_slider.val), T2, int(slice_slider.val)))
    fig.canvas.draw_idle()
FA_right_button.on_clicked(increase_FA)

# Create a `matplotlib.widgets.Button` to decrease the FA slider value (by 2)
FA_left_button_ax = plt.axes([0.286, 0.03, 0.014, 0.03])
FA_left_button = Button(FA_left_button_ax, '<', color=axcolor, hovercolor='0.975')
def decrease_FA(event):
    if FA_slider.val in range(12,181):
        FA_slider.set_val(FA_slider.val-2)
        simulator.set_data(f(PD, int(TR_slider.val), T1, FA_slider.val, int(TE_slider.val), T2, int(slice_slider.val)))
    elif FA_slider.val==11:
        FA_slider.set_val(10)
        simulator.set_data(f(PD, int(TR_slider.val), T1, 10, int(TE_slider.val), T2, int(slice_slider.val)))
    fig.canvas.draw_idle()
FA_left_button.on_clicked(decrease_FA)

# Create a `matplotlib.widgets.Button` to increase the TR slider value (by 20 each time)
TR_right_button_ax = plt.axes([0.7, 0.07, 0.014, 0.03])
TR_right_button = Button(TR_right_button_ax, '>', color=axcolor, hovercolor='0.975')
def increase_TR(event):
    if TR_slider.val in range(1,1981):
        TR_slider.set_val(TR_slider.val+20)
        simulator.set_data(f(PD, int(TR_slider.val), T1, FA_slider.val, int(TE_slider.val), T2, int(slice_slider.val)))
    elif TR_slider.val in range(1981,2000):
        TR_slider.set_val(2000)
        simulator.set_data(f(PD, 2000, T1, FA_slider.val, int(TE_slider.val), T2, int(slice_slider.val)))
    fig.canvas.draw_idle()
TR_right_button.on_clicked(increase_TR)

# Create a `matplotlib.widgets.Button` to decrease the TR slider value (by 20)
TR_left_button_ax = plt.axes([0.286, 0.07, 0.014, 0.03])
TR_left_button = Button(TR_left_button_ax, '<', color=axcolor, hovercolor='0.975')
def decrease_TR(event):
    if int(TR_slider.val) > int(TE_slider.val)+1:
        if TR_slider.val in range(21,2001):
            TR_slider.set_val(TR_slider.val-20)
            simulator.set_data(f(PD, int(TR_slider.val), T1, FA_slider.val, int(TE_slider.val), T2, int(slice_slider.val)))
        elif TR_slider.val in range(1,21):
            TR_slider.set_val(1)
            simulator.set_data(f(PD, 1, T1, FA_slider.val, int(TE_slider.val), T2, int(slice_slider.val)))
    fig.canvas.draw_idle()
TR_left_button.on_clicked(decrease_TR)

# Create a `matplotlib.widgets.Button` to increase the TE slider value (by 1)
TE_right_button_ax = plt.axes([0.7, 0.11, 0.014, 0.03])
TE_right_button = Button(TE_right_button_ax, '>', color=axcolor, hovercolor='0.975')
def increase_TE(event):
    if TE_slider.val in range(1,100):
        TE_slider.set_val(TE_slider.val+1)
        simulator.set_data(f(PD, int(TR_slider.val), T1, FA_slider.val, int(TE_slider.val), T2, int(slice_slider.val)))
    fig.canvas.draw_idle()
TE_right_button.on_clicked(increase_TE)

# Create a `matplotlib.widgets.Button` to decrease the TE slider value (by 1)
TE_left_button_ax = plt.axes([0.286, 0.11, 0.014, 0.03])
TE_left_button = Button(TE_left_button_ax, '<', color=axcolor, hovercolor='0.975')
def decrease_TE(event):
    if TE_slider.val in range(2,101):
        TE_slider.set_val(TE_slider.val-1)
        simulator.set_data(f(PD, int(TR_slider.val), T1, FA_slider.val, int(TE_slider.val), T2, int(slice_slider.val)))
    fig.canvas.draw_idle()
TE_left_button.on_clicked(decrease_TE)

# Create a `matplotlib.widgets.Button` to increase the slice slider value (by 2)
slice_right_button_ax = plt.axes([0.7, 0.15, 0.014, 0.03])
slice_right_button = Button(slice_right_button_ax, '>', color=axcolor, hovercolor='0.975')
def increase_slice(event):
    if slice_slider.val in range(1,179):
        slice_slider.set_val(slice_slider.val+2)
        simulator.set_data(f(PD, int(TR_slider.val), T1, FA_slider.val, int(TE_slider.val), T2, int(slice_slider.val)))
    elif slice_slider.val in range(179,180):
        slice_slider.set_val(180)
        simulator.set_data(f(PD, int(TR_slider.val), T1, FA_slider.val, int(TE_slider.val), T2, 180))
    fig.canvas.draw_idle()
slice_right_button.on_clicked(increase_slice)

# Create a `matplotlib.widgets.Button` to decrease the slice slider value (by 2)
slice_left_button_ax = plt.axes([0.286, 0.15, 0.014, 0.03])
slice_left_button = Button(slice_left_button_ax, '<', color=axcolor, hovercolor='0.975')
def decrease_slice(event):
    if slice_slider.val in range(3,181):
        slice_slider.set_val(slice_slider.val-2)
        simulator.set_data(f(PD, int(TR_slider.val), T1, FA_slider.val, int(TE_slider.val), T2, int(slice_slider.val)-2))
    elif slice_slider.val in range(1,3):
        slice_slider.set_val(1)
        simulator.set_data(f(PD, int(TR_slider.val), T1, FA_slider.val, int(TE_slider.val), T2, 1))
    fig.canvas.draw_idle()
    fig.canvas.draw_idle()
slice_left_button.on_clicked(decrease_slice)

# Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')
def reset(event):
    TR_slider.reset()
    TE_slider.reset()
    slice_slider.reset()    
button.on_clicked(reset)

# Create a `matplotlib.widgets.Button` to re-window the image.
rewindowax = plt.axes([0.8, 0.1, 0.1, 0.04])
rewindow_button = Button(rewindowax, 'Re-window', color=axcolor, hovercolor='0.975')
def rewindow(event):
    Mxy = f(PD, int(TR_slider.val), T1, int(FA_slider.val), int(TE_slider.val), T2, int(slice_slider.val))
    Mxy_sorted = np.sort(Mxy[Mxy[:]>0]) # don't take zeros into account
    low_5_pct = Mxy_sorted[int(np.round(0.05*len(Mxy_sorted)))] # low 5% level
    bkgr = np.mean(Mxy_sorted[Mxy_sorted<=low_5_pct], axis=0) # mean background level
    Mxy_no_bkgr = Mxy_sorted[Mxy_sorted>=10*bkgr]      #cut-off at 10x bkgr
    window_low  = Mxy_no_bkgr[int(round(0.10*len(Mxy_no_bkgr)))]  # low 5% level
    window_high = Mxy_no_bkgr[int(round(0.95*len(Mxy_no_bkgr)))]  # high 5% level
    Mxy = 64*(Mxy-window_low)/(window_high-window_low)
    simulator.set_data(Mxy)
    fig.canvas.draw_idle()
rewindow_button.on_clicked(rewindow)

plt.show()